<a href="https://colab.research.google.com/github/vyshali2419/NLP-2403A52028/blob/main/NLP%20LAB%20ASS%208(2403A52028%20BATCH%202).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!cat Lab8_NGram_Model_Vyshali_RollNo.ipynb

cat: Lab8_NGram_Model_Vyshali_RollNo.ipynb: No such file or directory


In [6]:
# Text preprocessing
import re
import string

# Tokenization and corpus utilities
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

# Counting and probability calculations
from collections import Counter, defaultdict

# Numerical operations
import numpy as np


In [7]:
# Sample dataset (replace with your own 1500+ word corpus if needed)
corpus = """
Natural language processing enables computers to understand human language.
Language models are widely used in speech recognition and text generation.
N-gram models predict the next word based on previous words.
Machine learning and NLP are important fields of artificial intelligence.
Text data is growing rapidly due to social media and digital communication.
"""

# Display sample text
print("Sample Text:\n")
print(corpus[:300])


Sample Text:


Natural language processing enables computers to understand human language.
Language models are widely used in speech recognition and text generation.
N-gram models predict the next word based on previous words.
Machine learning and NLP are important fields of artificial intelligence.
Text data is 


In [10]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Added to download the missing resource

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))

    sentences = sent_tokenize(text)
    processed_sentences = []

    for sent in sentences:
        words = word_tokenize(sent)
        words = [w for w in words if w not in stop_words]
        words = ['<s>'] + words + ['</s>']
        processed_sentences.append(words)

    return processed_sentences

processed_corpus = preprocess_text(corpus)
print(processed_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


[['<s>', 'natural', 'language', 'processing', 'enables', 'computers', 'understand', 'human', 'language', 'language', 'models', 'widely', 'used', 'speech', 'recognition', 'text', 'generation', 'ngram', 'models', 'predict', 'next', 'word', 'based', 'previous', 'words', 'machine', 'learning', 'nlp', 'important', 'fields', 'artificial', 'intelligence', 'text', 'data', 'growing', 'rapidly', 'due', 'social', 'media', 'digital', 'communication', '</s>']]


In [11]:
def build_ngram(sentences, n):
    ngrams = []
    for sent in sentences:
        for i in range(len(sent) - n + 1):
            ngrams.append(tuple(sent[i:i+n]))
    return Counter(ngrams)

# Build models
unigram = build_ngram(processed_corpus, 1)
bigram = build_ngram(processed_corpus, 2)
trigram = build_ngram(processed_corpus, 3)

print("Unigram sample:", list(unigram.items())[:5])
print("Bigram sample:", list(bigram.items())[:5])
print("Trigram sample:", list(trigram.items())[:5])


Unigram sample: [(('<s>',), 1), (('natural',), 1), (('language',), 3), (('processing',), 1), (('enables',), 1)]
Bigram sample: [(('<s>', 'natural'), 1), (('natural', 'language'), 1), (('language', 'processing'), 1), (('processing', 'enables'), 1), (('enables', 'computers'), 1)]
Trigram sample: [(('<s>', 'natural', 'language'), 1), (('natural', 'language', 'processing'), 1), (('language', 'processing', 'enables'), 1), (('processing', 'enables', 'computers'), 1), (('enables', 'computers', 'understand'), 1)]


In [13]:
vocab_size = len(unigram)

def laplace_probability(ngram, ngram_counts, prev_counts):
    return (ngram_counts[ngram] + 1) / (prev_counts + vocab_size)


In [14]:
def sentence_probability(sentence, n):
    words = preprocess_text(sentence)[0]
    prob = 1

    if n == 1:
        total = sum(unigram.values())
        for w in words:
            prob *= (unigram[(w,)] + 1) / (total + vocab_size)

    elif n == 2:
        for i in range(len(words)-1):
            bg = (words[i], words[i+1])
            prob *= laplace_probability(bg, bigram, unigram[(words[i],)])

    elif n == 3:
        for i in range(len(words)-2):
            tg = (words[i], words[i+1], words[i+2])
            prob *= laplace_probability(tg, trigram, bigram[(words[i], words[i+1])])

    return prob

sentences = [
    "language models are useful",
    "natural language processing is important",
    "machine learning uses data",
    "text generation is challenging",
    "artificial intelligence is growing"
]

for s in sentences:
    print(f"\nSentence: {s}")
    print("Unigram:", sentence_probability(s,1))
    print("Bigram:", sentence_probability(s,2))
    print("Trigram:", sentence_probability(s,3))



Sentence: language models are useful
Unigram: 1.4648437500000005e-08
Bigram: 8.22882722754353e-07
Trigram: 1.7756942964699194e-05

Sentence: natural language processing is important
Unigram: 4.882812500000002e-10
Bigram: 8.434273933050905e-08
Trigram: 1.7745268445984731e-06

Sentence: machine learning uses data
Unigram: 1.2207031250000005e-10
Bigram: 2.2750344161518885e-08
Trigram: 4.6728797275524195e-07

Sentence: text generation is challenging
Unigram: 7.324218750000001e-09
Bigram: 8.650818367417557e-07
Trigram: 1.7756942964699194e-05

Sentence: artificial intelligence is growing
Unigram: 9.765625000000004e-09
Bigram: 8.645130781377177e-07
Trigram: 1.7756942964699194e-05


In [15]:
def perplexity(sentence, n):
    words = preprocess_text(sentence)[0]
    N = len(words)
    prob = sentence_probability(sentence, n)
    return pow(1/prob, 1/N)

for s in sentences:
    print(f"\nSentence: {s}")
    print("Unigram Perplexity:", perplexity(s,1))
    print("Bigram Perplexity:", perplexity(s,2))
    print("Trigram Perplexity:", perplexity(s,3))



Sentence: language models are useful
Unigram Perplexity: 36.884316459269115
Bigram Perplexity: 16.479059097001397
Trigram Perplexity: 8.915102895753103

Sentence: natural language processing is important
Unigram Perplexity: 35.63594872561357
Bigram Perplexity: 15.100525103118121
Trigram Perplexity: 9.088374986375198

Sentence: machine learning uses data
Unigram Perplexity: 44.89848193237491
Bigram Perplexity: 18.78604412915197
Trigram Perplexity: 11.351917672117708

Sentence: text generation is challenging
Unigram Perplexity: 42.3689536419525
Bigram Perplexity: 16.315055716842274
Trigram Perplexity: 8.915102895753103

Sentence: artificial intelligence is growing
Unigram Perplexity: 40.00000000000001
Bigram Perplexity: 16.317201869996897
Trigram Perplexity: 8.915102895753103
